Let's test all operations in Pandas to check if it can compete with the sqlite implementation.

Object:
x: float, y: float, s: string

DataFrame Columns:
x, y, s, obj_id, obj
where obj_id is a unique index

Update: Nope, using an index to look up numbers is very bad. Building an indexed df of 10k items took > 1 second. 
Let's just resign ourselves to:
 - O(n) remove, maybe with a mark-and-sweep gc
 - O(n) update, since we have to find the obj_id.

O(n) update looking good! <1ms on 1M item set.

In [1]:
import random
import time
import pandas as pd
import duckdb
import sys
from pympler.asizeof import asizeof


In [2]:
letters = ['qwertyuiopasdfghjklzxcvbnm']
class Thing:
    def __init__(self):
        self.x = random.random()
        self.y = random.random()
        #self.s = ''.join(random.choice(letters) for _ in range(5))

n_things = 10**7
things = [Thing() for _ in range(n_things)]

# 10^7: 2.3G

In [3]:
con = duckdb.connect(database=':memory:')

In [4]:
t0 = time.time()
obj_lookup = {id(t): t for t in things}
t1 = time.time()
print('build dict', t1-t0)
# 2.9GB, so 600MB, yeowch

build dict 3.162687063217163


In [5]:
t1 = time.time()
values = [(t.x, t.y, id(t)) for t in things]
t2 = time.time()
con.execute('DROP TABLE IF EXISTS items')
con.execute("CREATE TABLE items(x DOUBLE, y DOUBLE, obj_id BIGINT)")
con.executemany("INSERT INTO items VALUES (?, ?, ?)", values)
t3 = time.time()
print('build dict', t2-t1)
print('insert all', t3-t2)

# mem total

setup 12.237265825271606
build dict 2.0264065265655518
insert all 314.59319376945496


In [6]:
print('build dict', t2-t1)
print('insert all', t3-t2)


build dict 2.0264065265655518
insert all 314.59319376945496


In [ ]:
t0 = time.time()
df = pd.DataFrame({
    'x': [t.x for t in things],
    'y': [t.y for t in things],
    'obj_id': [id(t) for t in things],
    }
)
t1 = time.time()
print(t1-t0)
# 10^7: , so 300MB. Nice!

In [ ]:
asizeof(df) / n_things

In [ ]:
# connect to an in-memory database
con = duckdb.connect()

# convert df into a DuckDB db. Before: 3.3GB
con.execute('DROP TABLE IF EXISTS my_table')
t0 = time.time()
con.execute("CREATE TABLE my_table AS SELECT * FROM df")
t1 = time.time()
print(t1-t0)
# Ater: 3.9GB, so 600MB used

In [ ]:
con.execute('SHOW TABLE my_table')
con.fetchall()

In [ ]:
con.execute('SELECT t from my_table LIMIT 10')
objs = con.fetchall()
print(objs[0][0])
# it does not actually store objects
# it's just storing 

In [ ]:
# query the Pandas DataFrame "my_df". Querying it through DuckDB is way slower.
thresh=0.0001
t0 = time.time()
resdd = con.execute("SELECT t FROM df where x < {} and y < 1".format(thresh)).df().t.to_list()
t1 = time.time()
respd = df.query('y <= 1 and x <= 0.0001'.format(thresh)).t.to_list()
t2 = time.time()

print('duckdb', len(resdd), t1-t0)
print('pandas', len(respd), t2-t1)


In [ ]:
# DuckDB querying pandas dfs is slower even with lazy eval.

rel = con.df(df)

t0 = time.time()
# chain together relational operators (this is a lazy operation, so the operations are not yet executed)
# equivalent to: SELECT i, j, i*2 as two_i FROM input_df ORDER BY i desc limit 2
transformed_rel = rel.filter(f'x < {thresh}')

# trigger execution by requesting .df() of the relation
# .df() could have been added to the end of the chain above - it was separated for clarity
res = transformed_rel.df().t.to_list()
t1 = time.time()
print(len(res), t1-t0)

In [ ]:
n_runs = 10
for e in range(0, 8):
    thresh = 10**e / len(df)
    t_duck = 0
    t_pd = 0
    for _ in range(n_runs):
        t0 = time.time()
        con.execute("SELECT t FROM my_table where y < 1 and x < {}".format(thresh))
        res = con.fetchall()
        t1 = time.time()
        res_pd = df.query('y <= 1 and x <= {}'.format(thresh)).t.to_list()
        t2 = time.time()
        t_duck += (t1-t0)/n_runs
        t_pd += (t2-t1)/n_runs
    # print(t_duck)
    print(len(res), len(res_pd), 'duckdb took {}x as long as pd.query'.format(round(t_duck / t_pd, 3)))

In [ ]:
t0 = time.time()
con.execute("SELECT obj_id, t FROM my_table")
all_objs = con.fetchall()
t1 = time.time()
print('got {} objs in {}'.format(len(all_objs), t1-t0))

In [ ]:
# update speed
ttot = 0
for _ in range(10):
    rand_id = random.choice(all_objs)[0]
    t0 = time.time()
    con.execute("UPDATE my_table SET x=12 where obj_id={}".format(rand_id))
    t1 = time.time()
    ttot += (t1-t0)/10
print(ttot)

In [ ]:
# delete speed
ttot = 0
for _ in range(10):
    del_id = random.choice(all_objs)[0]
    t0 = time.time()
    con.execute("DELETE FROM my_table where obj_id={}".format(del_id))
    t1 = time.time()
    ttot += (t1-t0)/10
print(ttot)

In [ ]:
con.execute("SELECT obj_id, t FROM my_table")
all_objs = con.fetchall()
print(len(all_objs))


In [ ]:
n_runs = 10
for e in range(3, 7):
    thresh = 10**e / len(df)
    t_duck = 0
    t_pd = 0
    for _ in range(n_runs):
        t0 = time.time()
        con.execute("SELECT t FROM my_table where x < {} and y < {}".format(thresh, thresh))
        res = con.fetchall()
        t1 = time.time()
        res_pd = df.query('y <= {} and x <= {}'.format(thresh, thresh)).t.to_list()
        t2 = time.time()
        t_duck += (t1-t0)/n_runs
        t_pd += (t2-t1)/n_runs
    print(len(res), len(res_pd), 'duckdb took {}x as long as pd.query'.format(round(t_duck / t_pd, 3)))

In [ ]:
"""
# 128 bytes / obj before
df = df.set_index('obj_id')
# 362 bytes / obj after, ouch.

n_lookups = 10**3
t0 = time.time()
for _ in range(n_lookups):
    oid = id(random.choice(things))
    df.loc[oid]
t1 = time.time()
print(t1-t0)
"""
# sub-millisecond find by id. That's good!

In [ ]:
# no-index lookup by id
n_runs = 10**3
t0 = time.time()
for _ in range(n_runs):
    t = random.choice(things)
    df[df['obj_id'] == id(t)]
t1 = time.time()
print((t1-t0)/n_runs)
t = random.choice(things)
df[df['obj_id'] == id(t)]

# still sub-ms lookup at 1M items, very nice!

In [ ]:
n_runs = 5
for e in range(0,8):
    n_finds = 10**e
    thresh = n_finds/10**7
    t_run = 0
    for _ in range(n_runs):
        df.query('y <= 1 and x <= {}'.format(thresh)).t.to_list()
        t0 = time.time()
        ls = df.query('y <= 1 and x <= {}'.format(thresh)).t.to_list()
        t1 = time.time()
        t_run += (t1-t0)/n_runs
    print(n_finds, t_run)

# 3.5ms query @ 1M, 38ms @ 10M

In [ ]:
n_runs = 10**2
t0 = time.time()
for _ in range(n_runs):
    t = random.choice(things)
    idx = df[df['obj_id'] == id(t)].index[0]
    df.at[idx,'x'] = 12
    t.x = 12
t1 = time.time()
print((t1-t0)/n_runs)
df.loc[idx]

# 5ms update on one of 10M items

In [ ]:
print(len(df))
t0=time.time()
df.drop(idx, inplace=True)
t1 = time.time()
print(len(df))
print(t1-t0)

In [ ]:
t1-t0